In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Flatten, Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.losses import binary_crossentropy
from keras.callbacks import ModelCheckpoint, EarlyStopping
import cv2
import serial
import json
import time


class SensorReader:
    def __init__(self, port='#apne laptop ka port', baudrate=9600):
        self.port = port
        self.baudrate = baudrate
        self.serial_connection = None

    def connect(self):
        try:
            self.serial_connection = serial.Serial(self.port, self.baudrate, timeout=1)
            return True
        except:
            return False

    def read_sensor_data(self):
        if not self.serial_connection:
            return np.array([70, 120, 80, 98, 16, 36.7])

        try:
            if self.serial_connection.in_waiting > 0:
                raw_data = self.serial_connection.readline().decode('utf-8').strip()
                parts = raw_data.split(',')
                if len(parts) >= 7:
                    return np.array([
                        float(parts[1]),  # heart_rate
                        float(parts[2]),  # bp_sys
                        float(parts[3]),  # bp_dia
                        float(parts[4]),  # oxygen_sat
                        float(parts[5]),  # resp_rate
                        float(parts[6])   # temp
                    ])
        except:
            pass

        return np.array([70, 120, 80, 98, 16, 36.7])

    def close(self):
        if self.serial_connection:
            self.serial_connection.close()

# Initialize sensor
sensor = SensorReader()
sensor.connect()
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False)
for layer in base_model.layers:
    layer.trainable = False
image_input = base_model.input
image_features = Flatten()(base_model.output)
image_features = Dense(128, activation='relu')(image_features)

# Biometric branch (6 sensor inputs)
biometric_input = Input(shape=(6,))  # HR, BP_sys, BP_dia, O2_sat, Resp_rate, Temp
biometric_features = Dense(32, activation='relu')(biometric_input)
biometric_features = Dense(16, activation='relu')(biometric_features)

# Combine branches
combined = Concatenate()([image_features, biometric_features])
combined = Dense(64, activation='relu')(combined)
output = Dense(1, activation='sigmoid')(combined)

model = Model(inputs=[image_input, biometric_input], outputs=output)
model.compile(optimizer='adam', loss=binary_crossentropy, metrics=['accuracy'])

# Data preparation
!wget -O train.zip "https://www.dropbox.com/scl/fi/qtcrzmsv4jz47cgvr2t0u/train.zip?rlkey=x3sz1h2o797inkqer67q7dg6b&dl=1"
!unzip train.zip

original_path = "/content/train"
new_path = "/content/medical_train"
import os
os.makedirs(new_path, exist_ok=True)
os.makedirs(f"{new_path}/NOT_SERIOUS", exist_ok=True)
os.makedirs(f"{new_path}/SERIOUS", exist_ok=True)

not_serious_emotions = ['Happy', 'Neutral', 'Surprise']
serious_emotions = ['Sad', 'Fear', 'Angry', 'Disguist']

import shutil
for emotion in not_serious_emotions:
    src_folder = f"{original_path}/{emotion}"
    if os.path.exists(src_folder):
        for file in os.listdir(src_folder):
            src_file = f"{src_folder}/{file}"
            dst_file = f"{new_path}/NOT_SERIOUS/{emotion}_{file}"
            shutil.copy2(src_file, dst_file)

for emotion in serious_emotions:
    src_folder = f"{original_path}/{emotion}"
    if os.path.exists(src_folder):
        for file in os.listdir(src_folder):
            src_file = f"{src_folder}/{file}"
            dst_file = f"{new_path}/SERIOUS/{emotion}_{file}"
            shutil.copy2(src_file, dst_file)

print(f"NOT_SERIOUS: {len(os.listdir(f'{new_path}/NOT_SERIOUS'))} images")
print(f"SERIOUS: {len(os.listdir(f'{new_path}/SERIOUS'))} images")

# Simple data generator for training
def generate_training_data(image_gen, batch_size):
    while True:
        img_batch, labels = next(image_gen)

        bio_batch = []
        for label in labels:
            if label == 1:  # Serious emotion
                hr = np.random.normal(85, 10)
                bp_sys = np.random.normal(135, 15)
                bp_dia = np.random.normal(85, 10)
                o2_sat = np.random.normal(94, 4)
                resp_rate = np.random.normal(22, 5)
                temp = np.random.normal(37.8, 1)
            else:  # Not serious
                hr = np.random.normal(70, 8)
                bp_sys = np.random.normal(120, 12)
                bp_dia = np.random.normal(80, 8)
                o2_sat = np.random.normal(98, 2)
                resp_rate = np.random.normal(16, 3)
                temp = np.random.normal(36.7, 0.5)

            bio_batch.append([
                max(50, min(120, hr)),
                max(90, min(180, bp_sys)),
                max(60, min(110, bp_dia)),
                max(80, min(100, o2_sat)),
                max(8, min(40, resp_rate)),
                max(35, min(42, temp))
            ])

        yield [img_batch, np.array(bio_batch)], labels

# Setup data generators
train_datagen = ImageDataGenerator(
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    rescale=1./255
)

train_gen = train_datagen.flow_from_directory(
    directory=new_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_gen = val_datagen.flow_from_directory(
    directory=new_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

#combined generators
train_data = generate_training_data(train_gen, 32)
val_data = generate_training_data(val_gen, 32)

# Training setup
es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=5, verbose=1)
mc = ModelCheckpoint(filepath="best_model_sensor.h5", monitor='val_accuracy', verbose=1, save_best_only=True)

# Train model
history = model.fit(
    train_data,
    steps_per_epoch=10,
    epochs=30,
    validation_data=val_data,
    validation_steps=8,
    callbacks=[es, mc]
)

# Real-time prediction function
def predict_with_sensors(image_path):
    # Load and preprocess image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    # Get real-time sensor data
    sensor_data = sensor.read_sensor_data()
    sensor_data = np.expand_dims(sensor_data, axis=0)

    # Make prediction
    prediction = model.predict([img, sensor_data])

    emotion_state = "SERIOUS" if prediction[0][0] > 0.5 else "NOT_SERIOUS"
    confidence = prediction[0][0] if prediction[0][0] > 0.5 else 1 - prediction[0][0]

    return {
        'emotion_state': emotion_state,
        'confidence': float(confidence),
        'heart_rate': float(sensor_data[0][0]),
        'bp_systolic': float(sensor_data[0][1]),
        'bp_diastolic': float(sensor_data[0][2]),
        'oxygen_saturation': float(sensor_data[0][3]),
        'respiratory_rate': float(sensor_data[0][4]),
        'temperature': float(sensor_data[0][5])
    }

# monitoring function
def monitor_patient(image_path):
    result = predict_with_sensors(image_path)

    print(f"Patient Status: {result['emotion_state']}")
    print(f"Confidence: {result['confidence']:.2f}")
    print(f"Heart Rate: {result['heart_rate']:.0f} bpm")
    print(f"Blood Pressure: {result['bp_systolic']:.0f}/{result['bp_diastolic']:.0f} mmHg")
    print(f"Oxygen Saturation: {result['oxygen_saturation']:.1f}%")
    print(f"Respiratory Rate: {result['respiratory_rate']:.0f} /min")
    print(f"Temperature: {result['temperature']:.1f}°C")

    return result
